# 4장. 임베딩
컴퓨터는 자연어를 직접적으로 처리할 수 없음!  
수치연산만 가능해서 자연어를 숫자나 벡터 형태로 변환해야 하는데, 이런 과정을 `임베딩`이라고 함.  
즉, 단어나 문장을 수치화해 벡터 공간으로 표현하는 과정을 말함.  
임베딩 기법에는 `문장 임베딩`과 `단어 임베딩`이 있음!  


## 4.2. 단어 임베딩
`단어 임베딩`은 말뭉치에서 각각의 단어를 벡터로 변환하는 기법을 말함.  
단어 임베딩은 의미와 문법적 정보를 지니고 있음.  

### 4.2.1 원-핫 인코딩
`원-핫-인코딩`은 단어를 숫자 벡터로 변환하는 가장 기본적인 방법임.  
요소들 중 단 하나의 값만 1이고 나머지 요솟값은 0인 인코딩을 의미함.  
원-핫-인코딩으로 나온 결과를 `원-핫-벡터`라고 하고, 전체 요소 중 단 하나만 1이기 때문에 `희소 벡터`라고 함!  

In [ ]:
from konlpy.tag import Komoran
import numpy as np

komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

# 단어 사전 구축 및 단어별 인덱스 부여
dics = {}
for word in nouns:
    if word not in dics.keys():
        dics[word] = len(dics)
print(dics)

In [ ]:
komoran = Komoran()
text = "오늘 날씨는 구름이 많아요."

# 명사만 추출
nouns = komoran.nouns(text)
print(nouns)

# 단어 사전 구축 및 단어별 인덱스 부여
dics = {}
for word in nouns:
    if word not in dics.keys():
        dics[word] = len(dics)
print(dics)

# 원-핫-인코딩
nb_classes = len(dics)
targets = list(dics.values())
one_hot_targets = np.eye(nb_classes)[targets]
print(one_hot_targets)

### 4.2.2 희소 표현과 분산 표현
단어가 희소벡터로 표현되는 방식을 `희소 표현`이라고 한다.  
희소 표현은 각각의 차원이 독립적인 정보를 지니고 있어 사람이 이해하기에 직관적인 장점, 단어 간의 연관성이 전혀 없어 의미를 담을 수 없고, 단어 사전의 크기가 커질 수록 메모리 낭비와 계산 복잡도가 커지는 단점.  

자연어 처리를 잘하기 위해서는 기본 토큰이 되는 단어의 의미와 주변 단어 간의 관계가 단어 임베딩에 표현되어야 함.  
이를 해결하기 위해 단어 간의 유사성을 잘 표현하면서도 벡터 공간을 절약할 수 있는 방법을 고안했는데, 이를 `분산 표현`이라고 함! 

### 4.2.3 Word2Vec
원-핫-인코딩의 경우에는 구현이 간단하지만, 챗봇의 경우에는 단어간의 유사도를 계산할 수 있어야 좋은 성능을 낸다는 단점이 있다.  
그래서 챗봇의 경우에는 원-핫-인코딩은 좋은 기법은 아니다.  
그렇기 때문에! 분산 표현 형태의 단어 임베딩 모델을 사용할 것이다.  
대표적인 모델로는 `Word2Vec`모델이 있다.  

In [ ]:
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

# 네이버 영화 리뷰 데이터 읽어옴
def read_review_data(filename):
    with open('rating_test.txt', 'r') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:]
    return data

# 학습 시간 측정 시작
start = time.time()

# 리뷰 파일 읽어오기
print('1) 더미 데이터 읽기 시작')
review_data = read_review_data('./rating_test.txt')
print(len(review_data))
print('1) 더미 데이터 읽기 완료 : ', time.time() - start)

# 문장 단위로 명사만 추출해 학습 입력 데이터로 만들기
print('2) 형태소에서 명사만 추출 시작')
komoran = Komoran()
docs = [komoran.nouns(sentence[1]) for sentence in review_data]
print('2) 형태소에서 명사만 추출 완료 : ', time.time() - start)

# Word2Vec 모델 학습
print('3) Word2Vec 모델 학습 시작')
model = Word2Vec(sentences=docs, vector_size=200, window=4, hs=1, min_count=2, sg=1)
print('3) Word2Vec 모델 학습 완료 : ', time.time() - start)

# 모델 저장
print('4) 학습된 모델 저장 시작')
model.save('nvmc.model')
print('4) 학습된 모델 저장 완료 : ', time.time() - start)

# 학습된 말뭉치 수, 코퍼스 내 전체 단어 수
print("corpus_count: ", model.corpus_count)
print("corpus_total_words : ", model.corpus_total_words)

In [ ]:
from gensim.models import Word2Vec

# 모델 로딩
model = Word2Vec.load('nvmc.model')
print("corpus_total_words : ", model.corpus_total_words)

# '사랑'이란 단어로 생성한 단어 임베딩 벡터
print('사랑 : ', model.wv['사랑'])

# 단어 유사도 계산
print("일요일 = 월요일\t", model.wv.similarity(w1='일요일', w2='월요일'))
print("안성기 = 배우\t", model.wv.similarity(w1='안성기', w2='배우'))
print("대기업 = 삼성\t", model.wv.similarity(w1='대기업', w2='삼성'))
print("일요일 != 삼성\t", model.wv.similarity(w1='일요일', w2='삼성'))
print("히어로 != 삼성\t", model.wv.similarity(w1='히어로', w2='삼성'))



# 가장 유사한 단어 추출
print(model.wv.most_similar("안성기", topn=5))
print(model.wv.most_similar("시리즈", topn=5))